Clustering and Segmenting step1

*Importing Libraries

In [3]:
!pip install BeautifulSoup4
!pip install geopy
!pip install lxml
!pip3 install lxml

     |████████████████████████████████| 122kB 8.1MB/s eta 0:00:01
     |████████████████████████████████| 112kB 5.3MB/s eta 0:00:01
     |████████████████████████████████| 5.5MB 5.7MB/s eta 0:00:01     |██▊                             | 460kB 5.7MB/s eta 0:00:01     |██████████████████▉             | 3.2MB 5.7MB/s eta 0:00:01     |██████████████████████▏         | 3.8MB 5.7MB/s eta 0:00:01     |████████████████████████████▋   | 4.9MB 5.7MB/s eta 0:00:01
     |████████████████████████████████| 5.4 MB 4.9 MB/s eta 0:00:01


In [4]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe

import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [5]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, features="html.parser")
table = soup.find("table")
table_rows = table.tbody.find_all("tr")

res = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text for tr in td]
    
    # Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    if row != [] and row[1] != "Not assigned":
        # If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough.
        if "Not assigned" in row[2]: 
            row[2] = row[1]
        res.append(row)

# Dataframe with 3 columns
df = pd.DataFrame(res, columns = ["PostalCode", "Borough", "Neighborhood"])
df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [6]:
df["Neighborhood"] = df["Neighborhood"].str.replace("\n","")
df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned
1,M2A\n,Not assigned\n,Not assigned
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"


In [8]:
df = df.groupby(["PostalCode", "Borough"])["Neighborhood"].apply(", ".join).reset_index()
df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned
1,M1B\n,Scarborough\n,"Malvern, Rouge"
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
4,M1G\n,Scarborough\n,Woburn


Use the .shape method to print the number of rows of your dataframe.

In [9]:
print("Shape: ", df.shape)

Shape:  (180, 3)


Get the latitude and the longitude coordinates of each neighborhood

Use geocoder package

In [10]:
!pip install geocoder

     |████████████████████████████████| 102kB 12.1MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/d2/3d/fa76db83bf75c4f8d338c2fd15c8d33fdd7ad23a9b5e57eb6c5de26b430e/click-7.1.2-py2.py3-none-any.whl
     |████████████████████████████████| 829kB 7.7MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future


In [11]:
!pip install geopy

In [12]:
df1 = pd.read_csv("./Geospatial_Coordinates.csv")
df1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
df1.rename(columns={'Postal Code':'Postcode'},inplace=True)
df1.head()
#Cancatenation
frames=[df,df1]
df_toronto=pd.concat(frames, axis=1, sort=False)
# Merging the two columns on 'Postcode'


df_toronto.head()


,PostalCode,Borough,Neighborhood,Postcode,Latitude,Longitude
0,M1A\n,Not assigned\n,Not assigned,M1B,43.806686,-79.194353
1,M1B\n,Scarborough\n,"Malvern, Rouge",M1C,43.784535,-79.160497
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek",M1E,43.763573,-79.188711
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill",M1G,43.770992,-79.216917
4,M1G\n,Scarborough\n,Woburn,M1H,43.773136,-79.239476


In [18]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


In [19]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto

Assign Markers in map

In [21]:
for lat, lng, borough, neighborhood in zip(
        df1['Latitude'], 
        df1['Longitude'], 
        df_toronto['Borough'], 
        df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

In [22]:
df_toronto_denc = df_toronto[df_toronto['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_toronto_denc.head()



,PostalCode,Borough,Neighborhood,Postcode,Latitude,Longitude
0,M4E\n,East Toronto\n,The Beaches,M5N,43.711695,-79.416936
1,M4K\n,East Toronto\n,"The Danforth West, Riverdale",M5T,43.653206,-79.400049
2,M4L\n,East Toronto\n,"India Bazaar, The Beaches West",M5V,43.628947,-79.394420
3,M4M\n,East Toronto\n,Studio District,M5W,43.646435,-79.374846
4,M4N\n,Central Toronto\n,Lawrence Park,M5X,43.648429,-79.382280


In [23]:
map_toronto_denc = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, borough, neighborhood in zip(
        df1['Latitude'], 
        df1['Longitude'], 
        df_toronto_denc['Borough'], 
        df_toronto_denc['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_denc)  

map_toronto_denc